In [1]:
%matplotlib ipympl

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pykonal
import seispy

HOME         = os.environ['HOME']
GOOGLE_DRIVE = os.environ['GOOGLE_DRIVE']

In [2]:
ak135 = pd.read_csv(
    os.path.join(HOME, 'src/pykonal/pykonal/data/AK135F_AVG.csv'),
    header=None,
    names=('depth', 'density', 'vp', 'vs', 'Q_kappa', 'Q_mu')
)
ak135['radius'] = 6371 - ak135['depth']
ak135 = ak135.sort_values('radius')

iasp91 = pd.read_csv(
    os.path.join(HOME, 'src/pykonal/pykonal/data/IASP91.csv'),
    header=None,
    names=('depth', 'radius', 'vp', 'vs')
)

mit2008 = pd.read_csv(
    os.path.join(HOME, 'src/pykonal/pykonal/data/mitp2008.xzv'),
    header=None,
    names=('degrees', 'radius', 'dv'),
    delim_whitespace=True
)
mit2008['radians'] = np.pi - np.radians(mit2008['degrees'] + 16.5)
mit2008 = mit2008.groupby(['radians', 'radius']).mean().reset_index()

In [3]:
def rotation_matrix(*args):
    """
    Return the rotation matrix used to rotate a set of cartesian
    coordinates by alpha radians about the z-axis, then beta radians
    about the y'-axis and then gamma radians about the z''-axis.
    """
    if len(args) == 1:
        alpha, beta, gamma = args[0], 0, 0
    elif len(args) == 3:
        alpha, beta, gamma = args
    else:
        raise(ValueError("number of positional arguments must be 1 or 3"))
    aa = np.array([[np.cos(alpha), -np.sin(alpha), 0],
                        [np.sin(alpha), np.cos(alpha), 0],
                        [0, 0, 1]])
    bb = np.array([[np.cos(beta), 0, np.sin(beta)],
                        [0, 1, 0],
                        [-np.sin(beta), 0, np.cos(beta)]])
    cc = np.array([[np.cos(gamma), -np.sin(gamma), 0],
                        [np.sin(gamma), np.cos(gamma), 0],
                        [0, 0, 1]])
    return (aa.dot(bb).dot(cc))

def decorate(func, default):
    def wrapper(*args):
        try:
            return (func(*args))
        except Exception:
            return (default)
    return (wrapper)
    
def map_coords(src, src_cs, dest_cs, src_origin):
    '''
    Return the coordinates of src in the dest reference frame.
    
    :param pykonal.GridND src: Coordinate grid to transform.
    :param str src_cs: Coordinate system to transform from.
    :param str dest_cs: Coordinate system to transform to.
    :param src_origin: Coordinates of the origin of src w.r.t. the destination frame of reference.
    '''
    pgrid_src = src.pgrid[...].copy()
    if src_cs.lower() == 'spherical' and dest_cs.lower() == 'spherical':
        xx_src = pgrid_src[...,0] * np.sin(pgrid_src[...,1]) * np.cos(pgrid_src[...,2])
        yy_src = pgrid_src[...,0] * np.sin(pgrid_src[...,1]) * np.sin(pgrid_src[...,2])
        zz_src = pgrid_src[...,0] * np.cos(pgrid_src[...,1])
        src_origin_xyz = [
            src_origin[0] * np.sin(src_origin[1]) * np.cos(src_origin[2]),
            src_origin[0] * np.sin(src_origin[1]) * np.sin(src_origin[2]),
            src_origin[0] * np.cos(src_origin[1])
        ]
        xx_src_idf  = xx_src + src_origin_xyz[0]
        yy_src_idf  = yy_src + src_origin_xyz[1]
        zz_src_idf  = zz_src + src_origin_xyz[2]
        xyz_src_idf = np.moveaxis(np.stack([xx_src_idf,yy_src_idf,zz_src_idf]), 0, -1)
        
        rr_src_idf  = np.sqrt(np.sum(np.square(xyz_src_idf), axis=-1))
        old         = np.seterr(divide='ignore', invalid='ignore')
        tt_src_idf  = np.arccos(xyz_src_idf[...,2] / rr_src_idf)
        np.seterr(**old)
        pp_src_idf  = np.arctan2(xyz_src_idf[...,1], xyz_src_idf[...,0])
        rtp_src_idf = np.moveaxis(np.stack([rr_src_idf, tt_src_idf, pp_src_idf]), 0, -1)
        return (rtp_src_idf)
    elif src_cs.lower() == 'cartesian' and dest_cs.lower() == 'spherical':
        src_origin_xyz = [
            src_origin[0] * np.sin(src_origin[1]) * np.cos(src_origin[2]),
            src_origin[0] * np.sin(src_origin[1]) * np.sin(src_origin[2]),
            src_origin[0] * np.cos(src_origin[1])
        ]
        xyz_src     = src.pgrid[...].dot(rotation_matrix(np.pi/2-src_origin[2], 0, np.pi/2-src_origin[1]))
        xyz_src    += src_origin_xyz
        xyz_src     = xyz_src
        rr_src      = np.sqrt(np.sum(np.square(xyz_src), axis=-1))
        old         = np.seterr(divide='ignore', invalid='ignore')
        tt_src      = np.arccos(xyz_src[...,2] / rr_src)
        np.seterr(**old)
        pp_src      = np.arctan2(xyz_src[...,1], xyz_src[...,0])
        rr_src_idf  = rr_src
        tt_src_idf  = tt_src
        pp_src_idf  = pp_src
        rtp_src_idf = np.moveaxis(np.stack([rr_src_idf,tt_src_idf, pp_src_idf]), 0, -1)
        return (rtp_src_idf)
    else:
        raise (NotImplementedError())

In [4]:
int_field                      = pykonal.EikonalSolver()
int_field.mode                 = 'spherical'
int_field.vgrid.min_coords     = mit2008['radius'].min(), np.pi/2, mit2008['radians'].min()
int_field.vgrid.node_intervals = 45.18, -1, 0.00156968
int_field.vgrid.npts           = 64, 1, 1637
int_field.vv                   = np.zeros(int_field.vgrid.npts)
int_field.dv                   = np.zeros(int_field.vgrid.npts)
dvi                            = pykonal.LinearInterpolator3D(
    int_field.vgrid,
    mit2008.sort_values(['radius', 'radians'])['dv'].values.reshape(int_field.vgrid.npts)
)
for ir in range(int_field.vgrid.npts[0]):
    v0 = np.interp(int_field.vgrid[ir,0,0,0], ak135['radius'], ak135['vp'])
    for it in range(int_field.vgrid.npts[1]):
        for ip in range(int_field.vgrid.npts[2]):
            dv = dvi(int_field.vgrid[ir, it, ip])
            int_field.dv[ir, it, ip] = dv
            int_field.vv[ir, it, ip] = v0 * (1 + dv*1e-2)


near_field_origin               = (6031, np.pi/2, 25 * np.pi/32)
near_field                      = pykonal.EikonalSolver()
near_field.mode                 = 'spherical'
near_field.vgrid.min_coords     = 1, np.pi/2, 0
near_field.vgrid.node_intervals = 5, 1, np.pi/40
near_field.vgrid.npts           = 110, 1, 81
near_field.vv                   = np.zeros(near_field.vgrid.npts)
near_field.vv = np.apply_along_axis(
    decorate(pykonal.LinearInterpolator3D(int_field.vgrid, int_field.vv), 0),
    -1,
    map_coords(near_field, 'spherical', 'spherical', near_field_origin)
)

far_field_origin               = (6348, np.pi/2, 5 * np.pi/16)
far_field                      = pykonal.EikonalSolver()
far_field.mode                 = 'cartesian'
far_field.vgrid.min_coords     = -250, -250, 0
far_field.vgrid.node_intervals = 50, 10, 1
far_field.vgrid.npts           = 11, 26, 1
far_field.vv                   = np.zeros(far_field.vgrid.npts)
far_field.vv = np.apply_along_axis(
    decorate(pykonal.LinearInterpolator3D(int_field.vgrid, int_field.vv), 0),
    -1,
    map_coords(far_field, 'cartesian', 'spherical', far_field_origin)
)

In [5]:
for it in range(near_field.pgrid.npts[1]):
    for ip in range(near_field.pgrid.npts[2]):
        idx = (0, it, ip)
        near_field.uu[idx] = near_field.pgrid[idx + (0,)] / near_field.vv[idx]
        near_field.close.append(idx)
        near_field.is_far[idx] = False
near_field.solve()

rtp_int_inf = map_coords(
    int_field, 
    'spherical',
    'spherical', 
    (-near_field_origin[0], near_field_origin[1], near_field_origin[2])
)
rtp_int_inf[...,2] = np.mod(rtp_int_inf[...,2], 2*np.pi)

uui = decorate(pykonal.LinearInterpolator3D(near_field.pgrid, near_field.uu), np.nan)

shape = rtp_int_inf.shape
for ir in range(shape[0]):
    for it in range(shape[1]):
        for ip in range(shape[2]):
            idx = (ir, it, ip)
            u = uui(rtp_int_inf[idx])
            if not np.isnan(u):
                int_field.uu[idx] = u
                int_field.close.append(idx)
                int_field.is_far[idx]   = False
                int_field.is_alive[idx] = True
int_field.solve()

# uui = decorate(pykonal.LinearInterpolator3D(int_field.pgrid, int_field.uu), np.nan)
# rtp_far_iif = map_coords(
#     far_field, 
#     'cartesian',
#     'spherical', 
#     far_field_origin
# )

# for idx in np.argwhere(
#      (np.abs(rtp_far_iif[...,0]) <= int_field.pgrid.max_coords[0])
#     &(np.abs(rtp_far_iif[...,0]) >= int_field.pgrid.min_coords[0])
# ):
#     idx = tuple(idx)
#     u = uui(rtp_far_iif[idx])
#     if not np.isnan(u):
#         far_field.uu[idx] = u
#         far_field.close.append(idx)
#         far_field.is_far[idx]   = False
# far_field.solve()

In [6]:
rtp_near_iif = map_coords(near_field, 'spherical', 'spherical', near_field_origin)
xx_near = rtp_near_iif[...,0] * np.sin(rtp_near_iif[...,1]) * np.cos(rtp_near_iif[...,2])
yy_near = rtp_near_iif[...,0] * np.sin(rtp_near_iif[...,1]) * np.sin(rtp_near_iif[...,2])
zz_near = rtp_near_iif[...,0] * np.cos(rtp_near_iif[...,1])

xx_int = int_field.pgrid[...,0] * np.sin(int_field.pgrid[...,1]) * np.cos(int_field.pgrid[...,2])
yy_int = int_field.pgrid[...,0] * np.sin(int_field.pgrid[...,1]) * np.sin(int_field.pgrid[...,2])
zz_int = int_field.pgrid[...,0] * np.cos(int_field.pgrid[...,1])

rtp_far_iif = map_coords(far_field, 'cartesian', 'spherical', far_field_origin)
xx_far = rtp_far_iif[...,0] * np.sin(rtp_far_iif[...,1]) * np.cos(rtp_far_iif[...,2])
yy_far = rtp_far_iif[...,0] * np.sin(rtp_far_iif[...,1]) * np.sin(rtp_far_iif[...,2])
zz_far = rtp_far_iif[...,0] * np.cos(rtp_far_iif[...,1])

plt.close('all')
fig = plt.figure(figsize=(8,6))
# fig = plt.figure(figsize=(32,24))
ax1 = fig.add_subplot(2, 1, 1, aspect=1)
ax2 = fig.add_subplot(2, 1, 2, aspect=1)

attr = 'uu'
vmin, vmax = getattr(int_field, attr).min(), getattr(int_field, attr).max()
# vmin, vmax = 0, 150

qmesh = ax1.pcolormesh(
    xx_int[:,0,:],
    yy_int[:,0,:],
    int_field.dv[:,it,:],
    vmin=-1,
    vmax=1,
    cmap=plt.get_cmap('seismic_r'),
    shading='gouraud'
)
ax1.scatter(
    near_field_origin[0] * np.sin(near_field_origin[1]) * np.cos(near_field_origin[2]),
    near_field_origin[0] * np.sin(near_field_origin[1]) * np.sin(near_field_origin[2]),
    marker='*',
    facecolors='none',
    edgecolor='k',
    s=500
)
cbar = fig.colorbar(qmesh, ax=ax1)
cbar.set_label(r'$\frac{dv}{v_P}$ [%]')
qmesh = ax2.pcolormesh(
    xx_int[:,0,:],
    yy_int[:,0,:],
    getattr(int_field, attr)[:,it,:],
    cmap=plt.get_cmap('jet_r'),
    vmin=vmin,
    vmax=vmax,
    shading='gouraud'
)
# pts = ax2.scatter(
#     xx_int[:,0,:],
#     yy_int[:,0,:],
#     c=getattr(int_field, attr)[:,it,:],
#     cmap=plt.get_cmap('jet_r'),
#     vmin=vmin,
#     vmax=vmax,
#     s=5,
#     linewidth=0
# )
# qmesh = pts
cbar = fig.colorbar(qmesh, ax=ax2)
cbar.set_label('Travel time [s]')
# ax2.scatter(
#     xx_near[:,0,:],
#     yy_near[:,0,:],
#     c=getattr(near_field, attr)[:,it,:],
#     cmap=plt.get_cmap('jet_r'),
#     vmin=vmin,
#     vmax=vmax,
#     s=10,
#     linewidth=0.1,
#     edgecolor='k'
# )
# ax.scatter(
#     xx_far[:,:,0],
#     yy_far[:,:,0],
#     c=far_field.uu[:,:,0],
#     cmap=plt.get_cmap('jet_r'),
#     vmin=vmin,
#     vmax=vmax,
#     s=rtp_far_iif[:,it,:,0]*1e-4,
# )
# cbar = fig.colorbar(pts, ax=ax, orientation='horizontal')

# fig = plt.figure()
# ax = fig.add_subplot(1, 1, 1, aspect=1)
# pts = ax.scatter(
#     xx_near[:,0,:],
#     yy_near[:,0,:],
#     c=near_field.uu[:,it,:],
#     cmap=plt.get_cmap('jet_r'),
#     s=rtp_near_iif[:,it,:,0]*1e-4,
# )
# cbar = fig.colorbar(pts, ax=ax, orientation='horizontal')

# fig = plt.figure()
# ax = fig.add_subplot(1, 1, 1, aspect=1)
# pts = ax.scatter(
#     xx_far[:,:,0],
#     yy_far[:,:,0],
#     c=far_field.uu[:,:,0],
#     cmap=plt.get_cmap('jet_r'),
#     s=rtp_far_iif[:,it,:,0]*1e-4,
# )
# cbar = fig.colorbar(pts, ax=ax, orientation='horizontal')

FigureCanvasNbAgg()